In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import plb
from plb.envs import make
import taichi as ti
# TODO: run on GPU, fast_math will cause error on float64's sqrt; removing it cuases compile error..
ti.init(arch=ti.gpu, debug=False, fast_math=True)

import numpy as np

In [ ]:
from plb.config import load
from plb.engine.taichi_env import TaichiEnv

In [ ]:
text = """
SIMULATOR:
  E: 5000.
  n_particles: 0  # This is unused.
  yield_stress: 200.
  quality: 1.0  # Can't make this lower :(
SHAPES:
  - shape: sphere
    radius: 0.2049069760770578/2
    init_pos: (0.6757143040494873, 0.5619162002773135, 0.7515980438048129)
    color: (127<<16)
    n_particles: 100
PRIMITIVES:
  - shape: Sphere
    radius: 0.03
    init_pos: (0.5757143040494873, 0.5619162002773135, 0.7515980438048129)
    color: (0.7, 0.7, 0.7)
    friction: 0.9
    action:
      dim: 3
      scale: (0.01, 0.01, 0.01)
  - shape: Sphere
    radius: 0.03
    init_pos: (0.7757143040494873, 0.5619162002773135, 0.7515980438048129)
    color: (0.7, 0.7, 0.7)
    friction: 0.9
    action:
      dim: 3
      scale: (0.01, 0.01, 0.01)
ENV:
  loss:
    target_path: envs/assets/Move3D-v1.npy
"""
cfg_path = "/tmp/taichi_cfg.yml"
with open(cfg_path, "w") as f:
    f.write(text.lstrip())
cfg = load(cfg_path)

In [ ]:
taichi_env = TaichiEnv(cfg=cfg, nn=False, loss=True)
taichi_env.initialize()
taichi_env.set_copy(False)
init_state = taichi_env.get_state()

In [ ]:
print(cfg.dump())

In [ ]:
import time

In [ ]:
taichi_env.set_state(**init_state)
state = taichi_env.get_state()
sim_state = state['state']
softness = 600
T = 5
action_dim = 3 * 2
action_max = 0.01

actions = np.random.random(size=(T, action_dim))*action_max

def forward(taichi_env, sim_state, action):
    dts = []
    taichi_env.set_state(sim_state, softness, False)
    # Timing is about ~20Hz.
    with ti.Tape(loss=taichi_env.loss.loss):
        for f in range(T):
            taichi_env.step(action[f])
            loss_info = taichi_env.compute_loss()
    loss = taichi_env.loss.loss[None]
    return loss, taichi_env.primitives.get_grad(len(action))

loss, grad = forward(taichi_env, sim_state, actions)

In [ ]:
print(loss)
print(grad.shape)

In [ ]:
# Check determinism - just seems like reduction?
loss0 = loss
for _ in range(10):
    loss, _ = forward(taichi_env, sim_state, actions)
    print(loss - loss0)